<a href="https://colab.research.google.com/github/kinome79/Colaborator/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
#  !pip install tf-nightly
#except Exception:
#  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.11.0


In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-02-28 18:42:58--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  2.03MB/s    in 0.2s    

2023-02-28 18:42:58 (2.03 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2023-02-28 18:42:58--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [4]:
# create datasets with the imported data
df_train = pd.read_csv(train_file_path, delimiter='\t', names=['type', 'message'])
df_test = pd.read_csv(test_file_path, delimiter='\t', names=['type', 'message'])

print ('Train DataFrame ----------------------------------------\n\n' + str(df_train))
print ('\nTest DataFrame ----------------------------------------\n\n' + str(df_test))


Train DataFrame ----------------------------------------

      type                                            message
0      ham  ahhhh...just woken up!had a bad dream about u ...
1      ham                           you can never do nothing
2      ham  now u sound like manky scouse boy steve,like! ...
3      ham  mum say we wan to go then go... then she can s...
4      ham  never y lei... i v lazy... got wat? dat day ü ...
...    ...                                                ...
4174   ham  just woke up. yeesh its late. but i didn't fal...
4175   ham  what do u reckon as need 2 arrange transport i...
4176  spam  free entry into our £250 weekly competition ju...
4177  spam  -pls stop bootydelious (32/f) is inviting you ...
4178   ham  tell my  bad character which u dnt lik in me. ...

[4179 rows x 2 columns]

Test DataFrame ----------------------------------------

      type                                            message
0      ham  i am in hospital da. . i will return home

In [5]:
# Verify no null values, and what unique values are in the type column

print('Are there any null values in df_train:', pd.isna(df_train).any().values)
print('Are there any null values in df_test:', pd.isna(df_test).any().values)
print('\nWhat are values for type in df_train:', df_train['type'].unique())
print('What are the values for type in df_test:', df_test['type'].unique())

categories = df_train['type'].unique()
print('\nLabels are 0=ham and 1=spam. Categories =', categories)

Are there any null values in df_train: [False False]
Are there any null values in df_test: [False False]

What are values for type in df_train: ['ham' 'spam']
What are the values for type in df_test: ['ham' 'spam']

Labels are 0=ham and 1=spam. Categories = ['ham' 'spam']


In [6]:
# Convert type to numeric 0/1 values

df_train['type'].replace(['ham','spam'], [0,1], inplace=True)
df_test['type'].replace(['ham','spam'], [0,1], inplace=True)

print (df_train)

      type                                            message
0        0  ahhhh...just woken up!had a bad dream about u ...
1        0                           you can never do nothing
2        0  now u sound like manky scouse boy steve,like! ...
3        0  mum say we wan to go then go... then she can s...
4        0  never y lei... i v lazy... got wat? dat day ü ...
...    ...                                                ...
4174     0  just woke up. yeesh its late. but i didn't fal...
4175     0  what do u reckon as need 2 arrange transport i...
4176     1  free entry into our £250 weekly competition ju...
4177     1  -pls stop bootydelious (32/f) is inviting you ...
4178     0  tell my  bad character which u dnt lik in me. ...

[4179 rows x 2 columns]


In [7]:
# Separate the data labels from the data
train_labels = df_train.pop('type').values
test_labels = df_test.pop('type').values

# Make the data an array instead of dataframe
train_data = df_train.values
test_data = df_test.values

print('Train Data ------------------------------------\n')
print(train_data)
print('\nTrain Label ---------------------------------------\n')
print(train_labels)

Train Data ------------------------------------

[['ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.']
 ['you can never do nothing']
 ['now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?']
 ...
 ['free entry into our £250 weekly competition just text the word win to 80086 now. 18 t&c www.txttowin.co.uk']
 ['-pls stop bootydelious (32/f) is inviting you to be her friend. reply yes-434 or no-434 see her: www.sms.ac/u/bootydelious stop? send stop frnd to 62468']
 ["tell my  bad character which u dnt lik in me. i'll try to change in  &lt;#&gt; . i ll add tat 2 my new year resolution. waiting for ur reply.be frank...good morning."]]

Train Label ---------------------------------------

[0 0 0 ... 1 1 0]


In [24]:
# Create Vocab Dict and functions to encode/decode strings 

import re

#create vocab variable to store all words, and their integer encoding
vocab = {'-':0} # make index 0 a non-used char... not sure if padding strings with 0's would effect if index 0 was a

# encode_text function - takes a string, and returns an array of numbers, adds to vocab dict 
def encode_text (mystring):

  #split string into array of lowercase words, removing punc but leaving @,!,$,?
  words = re.split(r'[., "\'(){}%&*+-]+', re.sub(r'([@!$?])', r' \1 ', mystring))
  nums = []

  # for each word, convert to int, adding to vocab if needed
  for word in words:
    word = word.lower()

    if word: # if not blank
      if word not in vocab:
        vocab[word] = len(vocab)
      nums.append(vocab[word])

  return nums


# decode_text function - takes array of numbers, returns array of words from vocab dict
def decode_text (myarray):

  # inverse the vocab dictionary and decode text
  inv_vocab = {val:key for key,val in vocab.items()}
  new_array = [inv_vocab[num] for num in myarray]

  return myarray


In [25]:
# Encode all strings to use in our analysis
train_data_encoded = [encode_text(arr[0]) for arr in train_data]
test_data_encoded = [encode_text(arr[0]) for arr in test_data]

print('Train Data Encoded ------------------------------------\n')
print(train_data_encoded[:5])
print('\nTest Data Encoded ------------------------------------\n')
print(test_data_encoded[:5])

Train Data Encoded ------------------------------------

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 11, 17, 18, 19, 14, 20, 21, 22, 10, 23, 24, 25, 14, 26, 27, 4, 28, 29], [30, 31, 32, 33, 34], [18, 11, 35, 16, 36, 37, 38, 39, 16, 5, 14, 40, 41, 42, 43, 44, 45, 46, 47, 11, 48, 49, 50, 51, 52, 53], [54, 55, 56, 57, 58, 59, 60, 59, 60, 61, 31, 62, 63, 64, 43, 65, 66], [32, 67, 68, 14, 69, 70, 71, 72, 53, 73, 74, 75, 76, 77, 43, 78, 79, 80, 81]]

Test Data Encoded ------------------------------------

[[14, 300, 82, 2661, 43, 14, 269, 1839, 45, 82, 2077], [301, 531, 2, 335, 7850, 111, 1991, 30, 53], [14, 845, 924, 138, 715, 203, 197, 288, 14, 775, 14, 247, 388, 6667, 111, 240, 112, 406, 53, 307, 11, 417, 77, 53], [6432, 231, 7, 7851, 7852, 46, 118, 775, 156, 14, 6223, 6432, 424, 892, 340, 98, 1226, 156, 197, 148, 5], [789, 229, 30, 1533, 53]]


In [44]:
# Find the average and maximum lenghts of the data strings to decide how the data should be padded

print("Average length of train strings is:", sum(len(arr) for arr in train_data_encoded)/len(train_data_encoded))
print("Max lengh of test strings is:", max(len(arr) for arr in train_data_encoded))
print("# of train strings  with len > 100 is:", sum(1 if len(arr) > 100 else 0 for arr in train_data_encoded))

print("\nAverage length of test strings is:", sum(len(arr) for arr in test_data_encoded)/len(test_data_encoded))
print("Max lengh of test strings is:", max(len(arr) for arr in test_data_encoded))
print("# of test strings  with len > 100 is:", sum(1 if len(arr) > 100 else 0 for arr in test_data_encoded))

Average length of train strings is: 16.704953338119168
Max lengh of test strings is: 190
# of train strings  with len > 100 is: 5

Average length of test strings is: 16.6558908045977
Max lengh of test strings is: 101
# of test strings  with len > 100 is: 1


In [45]:
# Padding sequences to the length of the 100, which will crop 6 entries but not have excessive padding

data_length = 100

train_data_encoded = keras.utils.pad_sequences(train_data_encoded, data_length)
test_data_encoded = keras.utils.pad_sequences(test_data_encoded, data_length)

print(test_data_encoded[:2])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   14  300   82 2661   43   14  269 1839   45
    82 2077]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0  301  531    2

In [23]:
# As I built my own vocab, need to determine how I want to set the input dim for the model (max possible vocab for a while)
print('Current number of words is', len(vocab))

Current number of words is 9199


In [51]:
# Create and compile model for analysis.

MAX_VOCAB_LENGTH = 10200  # Based on above numbers, this will allow addition of 1001 to vocab if needed

# Create the Sequential model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim = 10200, 
        mask_zero = True, 
        input_length = data_length,
        output_dim = 64),   
    tf.keras.layers.LSTM(64),       
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=['acc'])


In [52]:
# Train the model

history = model.fit(train_data_encoded, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
105/105 [==============================] - 19s 121ms/step - loss: 0.2738 - acc: 0.9189 - val_loss: 0.1013 - val_acc: 0.9785
Epoch 2/10
105/105 [==============================] - 10s 99ms/step - loss: 0.0686 - acc: 0.9812 - val_loss: 0.0576 - val_acc: 0.9868
Epoch 3/10
105/105 [==============================] - 11s 107ms/step - loss: 0.0393 - acc: 0.9895 - val_loss: 0.0601 - val_acc: 0.9844
Epoch 4/10
105/105 [==============================] - 10s 99ms/step - loss: 0.0266 - acc: 0.9934 - val_loss: 0.0637 - val_acc: 0.9797
Epoch 5/10
105/105 [==============================] - 10s 97ms/step - loss: 0.0160 - acc: 0.9973 - val_loss: 0.0491 - val_acc: 0.9880
Epoch 6/10
105/105 [==============================] - 10s 93ms/step - loss: 0.0131 - acc: 0.9961 - val_loss: 0.0452 - val_acc: 0.9904
Epoch 7/10
105/105 [==============================] - 10s 100ms/step - loss: 0.0063 - acc: 0.9994 - val_loss: 0.0625 - val_acc: 0.9856
Epoch 8/10
105/105 [==============================] - 11s 1

In [53]:
# Verify if the model was successful

results = model.evaluate(test_data_encoded, test_labels)
print(results)

44/44 [==============================] - 1s 27ms/step - loss: 0.0776 - acc: 0.9864
[0.07764226198196411, 0.9863505959510803]


In [67]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  
  encoded_text = keras.utils.pad_sequences( [encode_text(pred_text)], data_length )
  prediction = model.predict(encoded_text)
  prediction = [prediction[0][0], categories[round(prediction[0][0])]]


  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 32ms/step
[3.84491e-05, 'ham']


In [68]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 28ms/step
You passed the challenge. Great job!
